In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/yuncong/Brain/pipeline_scripts')
from utilities2014 import *
import os

os.environ['GORDON_DATA_DIR'] = '/home/yuncong/project/DavidData2014tif/'
os.environ['GORDON_REPO_DIR'] = '/home/yuncong/Brain'
os.environ['GORDON_RESULT_DIR'] = '/home/yuncong/project/DavidData2014results/'
os.environ['GORDON_LABELING_DIR'] = '/home/yuncong/project/DavidData2014labelings/'

dm = DataManager(data_dir=os.environ['GORDON_DATA_DIR'], 
  repo_dir=os.environ['GORDON_REPO_DIR'], 
  result_dir=os.environ['GORDON_RESULT_DIR'], 
  labeling_dir=os.environ['GORDON_LABELING_DIR'])

class args:
    stack_name = 'RS140'
    resolution = 'x5'
    slice_ind = 16
    gabor_params_id = 'blueNisslWide'
    segm_params_id = 'blueNisslRegular'
    vq_params_id = 'blueNissl'
    
dm.set_image(args.stack_name, args.resolution, args.slice_ind)
dm.set_gabor_params(gabor_params_id=args.gabor_params_id)
dm.set_segmentation_params(segm_params_id=args.segm_params_id)
dm.set_vq_params(vq_params_id=args.vq_params_id)

In [ ]:
# try:
#     features_rotated = dm.load_pipeline_result('features_rotated', 'npy')

# except Exception as e:

features = dm.load_pipeline_result('features', 'npy')

valid_features = features[:, dm.mask].T
n_valid = len(valid_features)

del features

def rotate_features(fs):
    features_tabular = fs.reshape((fs.shape[0], dm.n_freq, dm.n_angle))
    max_angle_indices = features_tabular.max(axis=1).argmax(axis=-1)
    features_rotated = np.reshape([np.roll(features_tabular[i], -ai, axis=-1) 
                               for i, ai in enumerate(max_angle_indices)], (fs.shape[0], dm.n_freq * dm.n_angle))

    return features_rotated, max_angle_indices

from joblib import Parallel, delayed

n_splits = 1000
features_rotated_tuple_list = Parallel(n_jobs=16)(delayed(rotate_features)(fs) for fs in np.array_split(valid_features, n_splits))
features_rotated = np.vstack([fr for fr, mai in features_rotated_tuple_list])
del valid_features

dm.save_pipeline_result(features_rotated, 'features_rotated', 'npy')

In [ ]:
max_angle_indices = np.concatenate([mai for fr, mai in features_rotated_tuple_list]).T
dm.save_pipeline_result(max_angle_indices, 'max_angle_indices', 'npy')
